In [1]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dataset import csv_creator
import glob
from logger import Logger
from dataset import Dataset
from train_cpu_struct import TrainModel

In [3]:
### Read all data from location
loc1 = 'Training_Context/*.csv'
loc2 = 'Training_Answer/*.txt'
training_data = csv_creator.data_creator(loc1=loc1,loc2=loc2)

In [4]:
### Preprocess dataset
train_set = TrainModel.preprocess(training_data['train'])
test_set = TrainModel.preprocess(training_data['test'])

In [11]:
### Initialize and train model
model_name1 = 'unsloth/Llama-3.2-3B-Instruct' #Llama
model_name2 = 'unsloth/DeepSeek-R1-Distill-Llama-8B' #Deepseek
trainingSave = TrainModel.train_model(model_name_=model_name1, dataset1 = train_set,
                                           dataset2 = test_set, seq_length=1024)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Starting training..., iters: 24
[WARNING] Some sequences are longer than 1024 tokens. The longest sentence 37575 will be truncated to 1024. Consider pre-splitting your data to save memory.
[WARNING] Some sequences are longer than 1024 tokens. The longest sentence 9315 will be truncated to 1024. Consider pre-splitting your data to save memory.
[WARNING] Some sequences are longer than 1024 tokens. The longest sentence 9916 will be truncated to 1024. Consider pre-splitting your data to save memory.
[WARNING] Some sequences are longer than 1024 tokens. The longest sentence 35922 will be truncated to 1024. Consider pre-splitting your data to save memory.
Iter 1: Val loss 0.893, Val took 136.007s
[WARNING] Some sequences are longer than 1024 tokens. The longest sentence 44327 will be truncated to 1024. Consider pre-splitting your data to save memory.
[WARNING] Some sequences are longer than 1024 tokens. The longest sentence 37640 will be truncated to 1024. Consider pre-splitting your data to

In [1]:
import glob
from logger import Logger
from mlx_lm import load, generate
from dataset import csv_creator

### Initialize model for inference
model_name1 = 'unsloth/Llama-3.2-3B-Instruct' #Llama
model_name2 = 'unsloth/DeepSeek-R1-Distill-Llama-8B' #Deepseek
adapters = './adapters'

In [2]:
!ls adapters/

adapter_config.json  adapters.safetensors


In [4]:
!mlx_lm.fuse --model {model_name1}

Loading pretrained model
Fetching 9 files: 100%|███████████████████████| 9/9 [00:00<00:00, 271573.64it/s]


In [6]:
!mlx_lm.convert --hf-path {'fused_model'} --mlx-path {'fused_model_4bit'} -q

[INFO] Loading
[INFO] Quantizing
[INFO] Quantized model with 4.501 bits per weight.


In [4]:
!mlx_lm.convert --hf-path {model_name1} --mlx-path {model_name1} -q

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/bin/mlx_lm.convert", line 8, in <module>
    sys.exit(main())
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/mlx_lm/convert.py", line 58, in main
    convert(**vars(args))
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/mlx_lm/utils.py", line 828, in convert
    raise ValueError(
ValueError: Cannot save to the path unsloth/Llama-3.2-3B-Instruct as it already exists. Please delete the file/directory or specify a new path to save to.


In [5]:
### Load file
location = input('Enter file location: ')
files = csv_creator.read_table(location)
prompt = f'Analyze the company from investor prespective. Financial data {files}'

In [8]:
### Load model and generate response
model, tokenizer = load('fused_model_4bit')
response = generate(model, tokenizer, prompt= prompt, verbose=True)

Prompt: Analyze the company from investor prespective. Financial data Financial Ratios for Gold Apr 23, 2001-10, 2006-10, 2007-10, 2008-10, 2009-10, 2010-10, 2011-10, 2012-10, 2013-10, 2014-10, 2015-10, 2016-10, 2017-10, 2018-10, 2019-10, 2020-10, 2021-10, 2022-10, 2023-10, Revenue USD Mil, 0.0, 0.0, 5420.0, 5774.0, 4481.0, 5444.0, 6615.0, 6858.0, 3894.0, 4048.0, 4038.0, 4202.0, 4472.0, 4914.0, 5163.0, 5339.0, 6319.0, 6848.0, 6833.0, Gross Margin %, 0.0, 0.0, 54.78, 55.35, 51.15, 53.82, 53.35, 52.55, 48.97, 48.81, 50.54, 52.28, 53.64, 54.54, 54.33, 53.14, 53.92, 54.35, 50.71, Operating Income USD Mil, 0.0, 0.0, 584.0, 795.0, 47.0, 566.0, 1071.0, 1119.0, 386.0, 419.0, 522.0, 615.0, 807.0, 904.0, 941.0, 846.0, 1347.0, 1618.0, 1350.0, Operating Margin %, 0.0, 0.0, 10.77, 13.77, 1.05, 10.4, 16.19, 16.32, 9.91, 10.35, 12.93, 14.64, 18.05, 18.4, 18.23, 15.85, 21.32, 23.63, 19.76, Net Income USD Mil, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 719.0, 1210.0, 125

In [8]:
### Load model and generate response
model, tokenizer = load(model_name1)
response = generate(model, tokenizer, prompt=prompt, max_tokens=1024, temp=0.5, verbose=True)

Prompt: Analyze the company from investor prespective. Financial data Financial Ratios for Gold Apr 23, 2001-10, 2006-10, 2007-10, 2008-10, 2009-10, 2010-10, 2011-10, 2012-10, 2013-10, 2014-10, 2015-10, 2016-10, 2017-10, 2018-10, 2019-10, 2020-10, 2021-10, 2022-10, 2023-10, Revenue USD Mil, 0.0, 0.0, 5420.0, 5774.0, 4481.0, 5444.0, 6615.0, 6858.0, 3894.0, 4048.0, 4038.0, 4202.0, 4472.0, 4914.0, 5163.0, 5339.0, 6319.0, 6848.0, 6833.0, Gross Margin %, 0.0, 0.0, 54.78, 55.35, 51.15, 53.82, 53.35, 52.55, 48.97, 48.81, 50.54, 52.28, 53.64, 54.54, 54.33, 53.14, 53.92, 54.35, 50.71, Operating Income USD Mil, 0.0, 0.0, 584.0, 795.0, 47.0, 566.0, 1071.0, 1119.0, 386.0, 419.0, 522.0, 615.0, 807.0, 904.0, 941.0, 846.0, 1347.0, 1618.0, 1350.0, Operating Margin %, 0.0, 0.0, 10.77, 13.77, 1.05, 10.4, 16.19, 16.32, 9.91, 10.35, 12.93, 14.64, 18.05, 18.4, 18.23, 15.85, 21.32, 23.63, 19.76, Net Income USD Mil, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 719.0, 1210.0, 125